# Credit Card Analytics 💳

Last tested version: <img src="https://img.shields.io/badge/Atoti-v0.8.13-blue">

Welcome to our credit card analytics notebook! Here we'll be using Atoti to analyze `5` million records worth of credit card sales transactions, joined together with a rich data model of users, loans, and retailer attributes. Let's see what insights we can generate!

**💡 Note:** Our credit card datasets come from [Kaggle](https://www.kaggle.com/datasets/ealtman2019/credit-card-transactions/data), feel free to check it out!

<div style="text-align: center;" ><a href="https://www.atoti.io/?utm_source=gallery&utm_content=credit-card-analytics" target="_blank" rel="noopener noreferrer"><img src="https://data.atoti.io/notebooks/banners/Discover-Atoti-now.png" alt="Try atoti"></a></div>

## 1. Import Libraries

First things first, let's import our libraries.

In [1]:
import atoti as tt
import pandas as pd
import time
from pprint import pprint

## 2. Instantiate Atoti Server and Link UI

Next, we'll spin up the Atoti server and link the session to our web-based Atoti UI.

💡 **Note**: https://docs.atoti.io/latest/api/atoti.Session.html#atoti.Session

In [2]:
# Create a Session object with corresponding parameters set
session = tt.Session(
    user_content_storage="./content_backup",
    port=9093,
    java_options=["-Xms1G", "-Xmx10G"],
)
session.link

http://localhost:9093

_Note_: This is the session's local URL: it may not be reachable if Atoti is running on another machine.

## 3. Load CSVs from S3 Into Atoti Tables 

From here on, we'll be loading the following CSVs from S3 into Atoti Tables:

* [Credit Card Transaction Data](#Credit-Card-Transaction-Data)
* [Credit Card Info Data](#Credit-Card-Info-Data)
* [User Data](#User-Data)
* [Retailer Data](#Retailer-Data)
* [Loan Data](#Loans-Data)

💡 **Note:** We should be mindful of the (numerical) columns that we intend to use as hierarchies instead of measures (e.g. a User ID does not need to be aggregated!). For this reason, we can explicitly set specific Atoti Table columns as desired data types when loading data into Atoti Tables. See https://docs.atoti.io/latest/api/atoti.Session.read_csv.html#atoti.Session.read_csv for more info.

### Credit Card Transaction Data

Here we use the `read_csv` function to simultaneously load our credit card transaction data and create a corresponding Atoti Table. We explicitly define column data types with the `types` parameter and set the `date_patterns` parameter to account for our `Datetime` column. All other columns and their data types for the loaded dataset are automatically inferred via the `read_csv` function.

In [3]:
# Use the `read_csv` function to read CSV from S3 into an Atoti Table
cc_sales_table = session.read_csv(
    "s3://data.atoti.io/notebooks/retail-banking/data/credit_card_transactions_processed_5MM.csv.gz",
    table_name="Sales Transactions",
    types={
        "Datetime": tt.type.LOCAL_DATE_TIME,
        "User": tt.type.STRING,
        "Card": tt.type.STRING,
        "Merchant Name": tt.type.STRING,
        "Zip": tt.type.STRING,
        "MCC": tt.type.STRING,
    },
    date_patterns={"Datetime": "yyyy-MM-dd HH:mm:ss"},
)
cc_sales_table.head()

,User,Card,Datetime,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,1158,1,2013-05-29 21:08:00,-12.6,Swipe Transaction,Merchant 57154,Aurora,CO,80013.0,5812,N/A,No
1,1458,0,2007-05-03 18:46:00,-15.412,Swipe Transaction,Merchant 57154,Ambler,PA,19002.0,5812,N/A,No
2,792,1,2015-11-28 16:08:00,1.672,Chip Transaction,Merchant 47076,Yonkers,NY,10703.0,5921,N/A,No
3,1765,0,2014-07-12 02:03:00,-1.82,Swipe Transaction,Merchant 13083,Chilton,WI,53014.0,5921,N/A,No
4,1158,1,2013-07-02 20:55:00,-15.944,Swipe Transaction,Merchant 57154,Aurora,CO,80013.0,5812,N/A,No


### Credit Card Info Data

Here we use the `read_csv` function to simultaneously load our credit card info data and create a corresponding Atoti Table. We explicitly define column data types with the `types` parameter, while all other columns and their data types for the loaded dataset are automatically inferred via the `read_csv` function.

In [ ]:
# Use the `read_csv` function to read CSV from S3 into an Atoti Table
user_cc_table = session.read_csv(
    "s3://data.atoti.io/notebooks/retail-banking/data/cards_processed.csv",
    table_name="User Credit Cards",
    types={
        "User": tt.type.STRING,
        "Card": tt.type.STRING,
        "Retailer ID": tt.type.STRING,
        "Card Number": tt.type.STRING,
        "CVV": tt.type.STRING,
    },
)
user_cc_table.head()

### User Data

Here we use the `read_csv` function to simultaneously load our user data and create a corresponding Atoti Table. We explicitly define column data types with the `types` parameter, while all other columns and their data types for the loaded dataset are automatically inferred via the `read_csv` function.

In [ ]:
# Use the `read_csv` function to read CSV from S3 into an Atoti Table
users_table = session.read_csv(
    "s3://data.atoti.io/notebooks/retail-banking/data/users_processed.csv",
    table_name="Users",
    types={
        "User": tt.type.STRING,
        "Birth Month": tt.type.STRING,
        "Zipcode": tt.type.STRING,
        "FICO Score": tt.type.STRING,
        "Current Age": tt.type.STRING,
        "Age Range": tt.type.STRING,
        "Retirement Age": tt.type.STRING,
    },
)
users_table.head()

### Retailer Data

Here we use the `read_csv` function to simultaneously load our retailer data and create a corresponding Atoti Table. We explicitly define column data types with the `types` parameter, while all other columns and their data types for the loaded dataset are automatically inferred via the `read_csv` function.

In [ ]:
# Use the `read_csv` function to read CSV from S3 into an Atoti Table
cc_info_table = session.read_csv(
    "s3://data.atoti.io/notebooks/retail-banking/data/retailers.csv",
    table_name="Credit Card Info",
    types={
        "Retailer ID": tt.type.STRING,
    },
)
cc_info_table.head()

### FICO Data

Here we use the `read_csv` function to simulatenously load our FICO data and create a corresponding Atoti Table. We explicitly define column data types with the `types` parameter, while all other columns and their data types for the loaded dataset are automatically inferred via the `read_csv` function.

In [ ]:
# Use the `read_csv` function to read CSV from S3 into an Atoti Table
fico_table = session.read_csv(
    "s3://data.atoti.io/notebooks/retail-banking/data/fico.csv",
    table_name="FICO",
    types={"FICO Score": tt.type.STRING},
)
fico_table.head()

### Loans Data

Here we use the `read_csv` function to simultaneously load our loans data and create a corresponding Atoti Table. We explicitly define column data types with the `types` parameter, while all other columns and their data types for the loaded dataset are automatically inferred via the `read_csv` function.

In [ ]:
# Use the `read_csv` function to read CSV from S3 into an Atoti Table
user_loans_table = session.read_csv(
    "s3://data.atoti.io/notebooks/retail-banking/data/loans.csv",
    types={
        "User": tt.type.STRING,
        "Inq Last 6mos": tt.type.STRING,
        "Delinq 2yrs": tt.type.STRING,
        "Public Record": tt.type.STRING,
        "Not Fully Paid": tt.type.STRING,
    },
    table_name="loans",
)
user_loans_table.head()

## 4. Join Tables and Create Cube

Now, that our data is loaded into our Atoti Tables and in the format that we want, we should join our tables together on common keys to extend attributes from other entity relationships, and create our Atoti cube.

In [ ]:
# Join tables
cc_sales_table.join(
    user_cc_table,
    (cc_sales_table["User"] == user_cc_table["User"])
    & (cc_sales_table["Card"] == user_cc_table["Card"]),
)
cc_sales_table.join(users_table, cc_sales_table["User"] == users_table["User"])
users_table.join(fico_table, users_table["FICO Score"] == fico_table["FICO Score"])
user_cc_table.join(
    cc_info_table, user_cc_table["Retailer ID"] == cc_info_table["Retailer ID"]
)
users_table.join(user_loans_table, users_table["User"] == user_loans_table["User"])

In [ ]:
# Create Cube from Atoti Table object
cube = session.create_cube(cc_sales_table)

## 5. Analyze the Atoti Cube

Now that the data has been loaded into the cube, we'll want to examine the data model of the cube to confirm that the columns and data types match up to what we expect. In addition, we can view the hierarchies, levels, and measures for our cube by assigning them to variables.

💡 **Note:** https://docs.atoti.io/latest/getting_started/tutorial/tutorial.html#Multidimensional-concepts

In [ ]:
# View the schema of the data model
session.tables.schema

In [ ]:
# Set variables for hierarchies, levels, and measures
h, l, m = cube.hierarchies, cube.levels, cube.measures

## 6. Enrich the Atoti Cube

Based on our analysis, there are a few aspects that we can do to enrich our cube, namely creating multi-level hierarchies, adjusting the ordering for particlar levels, and creating new measures.

* [Create Multi-Level Hierarchies](#Create-Multi-Level-Hierarchies)
* [Adjust Level Ordering](#Adjust-Level-Ordering)
* [Create New Measures](#Create-New-Measures)
* [Validate Measures](#Validate-Measures)
* [Create Credit Risk Measures](#Create-Credit-Risk-Measures)

### Create Multi-Level Hierarchies

Here we'll create a multi-level date hierarchy that allows us to break down our `Datetime` column into multiple levels. This allows us to drill-down from the top-level (in this case `Year`) to each underlying level, and see the breakdown of how each underlying level contributes to the overall measure aggregation. In addition, this also gives us more granularity in how we aggregate our measures, as we can specify the `OriginScope` which defines the level at which we perform our aggregations when creating new measures.

💡 **Note:** 

* https://docs.atoti.io/latest/api/atoti.Cube.create_date_hierarchy.html#atoti.Cube.create_date_hierarchy
* https://docs.atoti.io/latest/api/atoti.OriginScope.html#atoti.OriginScope

In [ ]:
# Create a multi-level date hierarchy
cube.create_date_hierarchy(
    "Transaction Date",
    column=cc_sales_table["Datetime"],
    levels={
        "Year": "yyyy",
        "Quarter": "QQQ",
        "Month": "MMM",
        "Day": "dd",
        "Hour": "HH",
    },
)

# Create multi-level hierarchy and clean up corresponding single-level hierarchies
h["Retailer Levels"] = [
    l["Level 1"],
    l["Level 2"],
    l["Level 3"],
    l["Level 4"],
    l["Level 5"],
]
del h["Level 1"]
del h["Level 2"]
del h["Level 3"]
del h["Level 4"]
del h["Level 5"]

### Adjust Level Ordering

In addition, we can use `CustomOrder` to customize the order of values for a specified level.

💡 **Note:** https://docs.atoti.io/latest/api/atoti.CustomOrder.html#atoti.CustomOrder

In [ ]:
# Set Income Range level ordering as a user-defined CustomOrder
l["Income Range"].order = tt.CustomOrder(
    first_elements=[
        "0K - 20K",
        "20K - 50K",
        "50K - 80K",
        "80K - 100K",
        "100K - 150K",
        "150K - 200K",
        "200K+",
    ]
)

# Set FICO level ordering as a user-defined CustomOrder
l["FICO Level"].order = tt.CustomOrder(
    first_elements=["Poor", "Fair", "Good", "Very Good", "Exceptional"]
)

### Create New Measures

Now, let's create some new measures! For this part, we'll be creating measures relating to user credit (credit cards) and debt (loans). Overall, we'll be showcasing a number of different aggregate functions, so feel free to check out the associated documentation as needed.

💡 **Note:** 

* https://docs.atoti.io/latest/api/atoti.agg.count_distinct.html#atoti.agg.count_distinct
* https://docs.atoti.io/latest/api/atoti.agg.single_value.html#atoti.agg.single_value
* https://docs.atoti.io/latest/api/atoti.agg.sum.html#atoti.agg.sum
* https://docs.atoti.io/latest/api/atoti.OriginScope.html#atoti.OriginScope



In [ ]:
# Create count distinct measure
m["Num Credit Cards"] = tt.agg.count_distinct(user_cc_table["Card Number"])

# Create single value measures
m["Income Annual"] = tt.agg.single_value(users_table["Yearly Income - Person"])
m["Interest Rate"] = tt.agg.single_value(user_loans_table["Interest Rate"])
m["Days with Credit Line"] = tt.agg.single_value(
    user_loans_table["Days with Credit Line"]
)
m["Total Debt.SV"] = tt.agg.single_value(users_table["Total Debt"])
m["Credit_Limit.SV"] = tt.agg.single_value(user_cc_table["Credit Limit"])
m["Revolving Balance.SV"] = tt.agg.single_value(user_loans_table["Revolving Bal"])
m["Revolving Utilization.SV"] = tt.agg.single_value(user_loans_table["Revol_Util"])

# Create measures and define aggregation scope (as needed)
m["Amount"] = tt.agg.sum(cc_sales_table["Amount"])
m["Total Debt"] = tt.agg.sum(
    m["Total Debt.SV"],
    scope=tt.OriginScope(levels={l[("Sales Transactions", "User", "User")]}),
)
m["Credit Limit"] = tt.agg.sum(
    m["Credit_Limit.SV"], scope=tt.OriginScope(levels={l["Card Number"]})
)
m["Revolving Balance"] = tt.agg.sum(
    m["Revolving Balance.SV"],
    scope=tt.OriginScope(levels={l["User"]}),
)
m["Revolving Utilization"] = tt.agg.sum(
    m["Revolving Utilization.SV"],
    scope=tt.OriginScope(levels={l[("Sales Transactions", "User", "User")]}),
)

# Create measures from other calculated measures
m["Utilization"] = m["Amount"] / m["Credit Limit"]
m["Utilization"].formatter = "DOUBLE[0.00%]"

### Validate Measures

We can then take a look at our newly created measures to validate that they are indeed working properly. Take a look at the underlying values to validate the measure calculations/aggregations.

In [ ]:
session.widget

### Create Credit Risk Measures


Lastly, we'll create new credit risk measures and calculate the expected credit loss for each user.

$$ ECL = EAD * PD * LGD $$

Where...
* ECL = Expected credit losses
* EAD = Exposure at default
* PD = Probability of default
* LGD = Loss given default

💡 **Note:** 

* https://docs.atoti.io/latest/api/atoti.agg.mean.html#atoti.agg.mean
* https://docs.atoti.io/latest/api/atoti.agg.sum_product.html#atoti.agg.sum_product

In [ ]:
# Create new credit risk measures
m["PD12"] = tt.agg.mean(
    tt.agg.single_value(users_table["PD12"]),
    scope=tt.OriginScope(levels={l[("Sales Transactions", "User", "User")]}),
)
m["PDLT"] = tt.agg.mean(
    tt.agg.single_value(users_table["PDLT"]),
    scope=tt.OriginScope(levels={l[("Sales Transactions", "User", "User")]}),
)
m["LGD"] = tt.agg.mean(
    tt.agg.single_value(users_table["LGD"]),
    scope=tt.OriginScope(levels={l[("Sales Transactions", "User", "User")]}),
)
m["EAD"] = m["Amount"]
m["ECL"] = tt.agg.sum_product(
    m["Amount"],
    m["LGD"],
    m["PD12"],
    scope=tt.OriginScope(levels={l[("Sales Transactions", "User", "User")]}),
)

# Format our measures as percentages
m["PD12"].formatter = "DOUBLE[0.00%]"
m["PDLT"].formatter = "DOUBLE[0.00%]"
m["LGD"].formatter = "DOUBLE[0.00%]"

# Categorize our measures
m["PD12"].folder = "Credit Risk"
m["PDLT"].folder = "Credit Risk"
m["LGD"].folder = "Credit Risk"
m["EAD"].folder = "Credit Risk"
m["ECL"].folder = "Credit Risk"

## 7. View Dashboards

With our cube fully set up and ready-to-go, let's now create our visualizations via slicing-and-dicing. We'll create dashboards for the following categories:

* [Overview](#Overview-Dashboard)
* [Customers](#Customers-Dashboard)
* [Retailers](#Retailers-Dashboard)
* [Transactions](#Transactions-Dashboard)

### Overview Dashboard

View overall credit risk, distribution of loan types, credit card exposure, etc.

In [ ]:
# Use `session.link` to link to Overview Dashboard
session.link / "#/dashboard/935"

### Customers Dashboard

View customer credit risk, loan types, credit card exposure, etc. 

In [ ]:
# Use `session.link` to link to Customers Dashboard
session.link / "#/dashboard/38c"

### Retailers Dashboard

View retailer exposure by industry.

In [ ]:
# Use `session.link` to link to Retailers Dashboard
session.link / "#/dashboard/3c2"

### Transactions Dashboard

View transactions by customer name, retailers, transaction year, or transaction city.

In [ ]:
# Use `session.link` to link to Transactions Dashboard
session.link / "#/dashboard/85d"

<div style="text-align: center;" ><a href="https://www.atoti.io/?utm_source=gallery&utm_content=credit-card-analytics" target="_blank" rel="noopener noreferrer"><img src="https://data.atoti.io/notebooks/banners/Your-turn-to-try-Atoti.jpg" alt="Try atoti"></a></div>